In [39]:
import numpy as np
from scipy.optimize import line_search

In [40]:
def rosenbrock(a, b, f0, n):
    def rosenbroke_calc(x):
        return sum(a * (x[i] ** 2 - x[i + 1]) ** 2 + b * (x[i] - 1) ** 2 for i in range(n - 1)) + f0
    return rosenbroke_calc

In [41]:
def rosenbrock_grad(a, b, f0, n):
    def rosenbrock_grad_calc(x):
        grad = []
        df_dx0 = 4 * a * x[0] * (x[0] ** 2 - x[1]) + 2 * b * (x[0] - 1)
        grad.append(df_dx0)
        for i in range(1, n - 1):
            df_dxi = 4 * a * x[i] * (x[i] ** 2 - x[i + 1]) + 2 * b * (x[i] - 1) - 2 * a * (x[i - 1] ** 2 - x[i])
            grad.append(df_dxi)
        df_dxn = - 2 * a * (x[n - 2] ** 2 - x[n - 1])
        grad.append(df_dxn)
        return np.array(grad)
    return rosenbrock_grad_calc

In [42]:
def rosenbrock_hessian(a, b, f0, n):
    def hessian_func(x):
        H = np.zeros((n, n))

        # Элементы вдоль главной диагонали
        for i in range(n):
            if i == 0:
                # Вторая производная по x0
                H[i, i] = 12 * a * x[0] ** 2 - 4 * a * x[1] + 2 * b
            elif i == n - 1:
                # Вторая производная по x_(n-1)
                H[i, i] = 2 * a
            else:
                # Вторая производная по x_i
                H[i, i] = 12 * a * x[i] ** 2 - 4 * a * x[i + 1] + 2 * b + 2 * a

        # Внедиагональные элементы (вторая производная по x_i и x_(i+1))
        for i in range(n - 1):
            H[i, i + 1] = -4 * a * x[i]
            H[i + 1, i] = H[i, i + 1]  # Симметричная матрица

        return H
    
    return hessian_func

In [43]:
def vector_norm(vec: np.ndarray) -> float:
    return max(map(abs, vec))

In [44]:
def find_min(func, a, b, xk, dk, eps=1e-4):
    phi = (1 + 5 ** 0.5) / 2
    a1 = b - (b - a) / phi
    b1 = a + (b - a) / phi

    while abs(b1 - a1) > eps:
        if func(xk + a1 * dk) < func(xk + b1 * dk):
            b = b1
        else:
            a = a1

        a1 = b - (b - a) / phi
        b1 = a + (b - a) / phi

    return (a + b) / 2

In [45]:
def grad_desc(func, func_grad, x0, M, epsilon1=1e-5, epsilon2=1e-5):
    xk = np.array(x0, dtype=float)
    k = 0
    
    while k < M:
        grad_fk = func_grad(xk)
        
        if np.linalg.norm(grad_fk) < epsilon1:
            break
        
        dk = -grad_fk
        alpha = line_search(func, func_grad, xk, dk, gfk=grad_fk)[0]

        xk_new = xk + alpha * dk
        
        if vector_norm(xk_new - xk) < epsilon1 and abs(func(xk_new) - func(xk)) < epsilon2:
            break
        
        xk = xk_new
        k += 1
        
    return xk, func(xk), k

In [46]:
def fletcher_reeves(func, func_grad, x0, M, eps=1e-5):
    dk = -func_grad(x0)
    xk = x0.copy()
    k = 0
    epsilon_div = 1e-10
    
    while k < M:
        grad_xk = func_grad(xk)
        if vector_norm(grad_xk) < eps:
            break
        
        alpha = find_min(func, -100, 100, xk, dk, 1e-4)
        xk1 = xk + alpha * dk
        
        grad_xk1 = func_grad(xk1)
        wk = (vector_norm(grad_xk1) ** 2) / (vector_norm(grad_xk) ** 2 + epsilon_div)
        
        dk = -grad_xk1 + wk * dk
        xk = xk1.copy()
        k += 1
    
    return xk, func(xk), k


In [47]:
def polack_ribeir(func, func_grad, x0, M, eps=1e-5):
    n = len(x0)
    dk = -func_grad(x0)
    xk = x0.copy()
    k = 0
    while k < M:
        grad_fk = func_grad(xk)
        if np.linalg.norm(grad_fk) < eps:
            return xk, func(xk), k

        alpha = find_min(func, -100, 100, xk, dk, 1e-4)
        xk1 = xk + alpha * dk
        if k % n == 0:
            wk = np.dot(func_grad(xk1), (func_grad(xk1) - func_grad(xk))) / (np.linalg.norm(grad_fk) ** 2)
        else:
            wk = 0
        dk = -func_grad(xk1) + wk * dk
        k += 1
        if k == n:
            xk = x0.copy()
        else:
            xk = xk1.copy()
        
    return xk, func(xk), k

In [48]:
def dav_fletch_pow(func, func_grad, x0, M, epsilon1=1e-5, epsilon2=1e-5): 
    Gk = np.eye(len(x0))
    xk = np.array(x0, dtype=float)
    k = 0
    while k < M:
        if np.linalg.norm(func_grad(xk)) < epsilon1:
            break
        dk = np.dot(-Gk, func_grad(xk))

        alpha = find_min(func, -1000, 1000, xk, dk, 1e-4)
            
        xk1 = xk + alpha * dk
        delta_xk = xk1 - xk
        delta_gk = func_grad(xk1) - func_grad(xk)
        delta_Gk = (np.outer(delta_xk, delta_xk.T) / (delta_xk.T @ delta_gk)) - ((np.outer(Gk @ delta_gk, delta_gk.T @ Gk.T)) / (delta_gk.T @ Gk @ delta_gk))
        Gk = Gk + delta_Gk
        xk = xk1
        k += 1
    return xk, func(xk), k

In [49]:
def levenb_marq(func, func_grad, hessian, x0, M, mu0=1e4, epsilon1=1e-5, epsilon2=1e-5):
    xk = np.array(x0, dtype=float)
    n = len(xk)
    mu_k = mu0
    k = 0
    
    while k < M:
        grad_fk = func_grad(xk)
        
        if np.linalg.norm(grad_fk) < epsilon1:
            return xk, func(xk), k
        
        Hk = hessian(xk)
        Hk_mu = Hk + mu_k * np.eye(n)
        Hk_mu_inv = np.linalg.inv(Hk_mu)
        
        dk = -np.dot(Hk_mu_inv, grad_fk)
        xk_new = xk + dk
        
        if func(xk_new) < func(xk):
            mu_k /= 2
        else:
            mu_k *= 2
        
        xk = xk_new
        k += 1
    
    return xk, func(xk), k

In [50]:
import pandas as pd

a, b, f0, n = 30, 2, 80, 4
M = 10000
x0 = np.random.uniform(0, 1, n)
func = rosenbrock(a, b, f0, n)
func_grad = rosenbrock_grad(a, b, f0, n)
H = rosenbrock_hessian(a, b, f0, n)
data = {
    'grad_desc': [],
    'fletcher_reeves': [],
    'polack_ribeir': [],
    'dav_fletch_pow': [],
    'levenb_marq': []
}
for eps in (1e-2, 1e-3, 1e-4, 1e-5):
    print(f'{eps = }')
    it_nums = []
    for method in (grad_desc, fletcher_reeves, polack_ribeir, dav_fletch_pow, levenb_marq):
        x, y, it_num = method(func, func_grad, x0, M, eps) if method.__name__ != 'levenb_marq' else method(func, func_grad, H, x0, M, eps)
        data[f'{method.__name__}'].append(it_num)
        print(method.__name__, x, y, it_num, sep='\n', end='\n\n')
        
df = pd.DataFrame(data=data, index=['1e-2', '1e-3', '1e-4', '1e-5'])
df.index.name = 'eps'


eps = 0.01
grad_desc
[0.99622901 0.99213379 0.98442236 0.9683333 ]
80.00065827563446
354

fletcher_reeves
[0.99980482 0.9996393  0.99919848 0.99832758]
80.00000198786256
45

polack_ribeir
[1.00017533 1.00035773 1.00072237 1.00146633]
80.00000137724935
58

dav_fletch_pow
[1.00000123 0.99998081 0.99995844 0.99989744]
80.00000002990333
14

levenb_marq
[1. 1. 1. 1.]
80.0
11

eps = 0.001
grad_desc
[0.99622901 0.99213379 0.98442236 0.9683333 ]
80.00065827563446
354

fletcher_reeves
[0.99999158 0.99997529 0.99994455 0.99987806]
80.00000001411564
107

polack_ribeir
[1.00007804 1.0001558  1.00031813 1.00064449]
80.00000026639741
81

dav_fletch_pow
[0.99999967 0.99999937 0.99999864 0.9999975 ]
80.00000000000644
15

levenb_marq
[1.00000002 1.00000003 1.00000006 1.00000011]
80.00000000000001
11

eps = 0.0001
grad_desc
[0.99767467 0.99514746 0.99037489 0.98037725]
80.00025107021439
418

fletcher_reeves
[0.99999872 0.99999728 0.99999512 0.99999109]
80.00000000009764
180
polack_ribeir
[1.00000109 1.0

In [51]:
df

,grad_desc,fletcher_reeves,polack_ribeir,dav_fletch_pow,levenb_marq
eps,,,,,
1e-2,354,45,58,14,11
1e-3,354,107,81,15,11
1e-4,418,180,128,15,11
1e-5,726,216,141,16,11
